# Task 3: Optimization

In the [Bin Packing Problem (BPP)](https://en.wikipedia.org/wiki/Bin_packing_problem), given a collection of items, the goal is to efficiently pack the items into the minimum number of bins, where each item has an associated weight, and the bins have a maximum weight. This problem is common in industries like supply chain management, where multiple packages need to be loaded onto a truck, plane, or vessel. In this task, you will solve the BPP using quantum computing.

In [ ]:
import numpy as np
import itertools
import time
from tqdm import tqdm

# Problem modeling imports
from docplex.mp.model import Model

# Qiskit imports
from qiskit import QuantumCircuit
from qiskit.circuit.library import TwoLocal, EfficientSU2
from qiskit.circuit import ParameterVector
from qiskit.circuit import Parameter
from qiskit_aer.primitives import Sampler
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms.utils import algorithm_globals
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.translators import from_docplex_mp
from qiskit_optimization.converters import QuadraticProgramToQubo

## 1) **From Integer Linear Programming (ILP) to Quadratic Unconstrained Binary Optimization (QUBO)**
   - Define the ILP formulation of the BPP. You can use [Docplex](https://qiskit-community.github.io/qiskit-optimization/tutorials/11_using_classical_optimization_solvers_and_models.html) or similar frameworks.
   - Create a function to transform the ILP model into a QUBO.
   - Test your function with specific instances (sizes: small, medium, and large).

In [3]:
# Define problem instances
smallest_weights = [2, 3]
small_weights = [2, 3, 4]
medium_weights = [2, 3, 4, 5, 6]
large_weights = [2, 3, 4, 5, 6, 7, 8, 9, 10]
bin_capacity = 5  # Maximum capacity of each bin

In [4]:
#  Define the ILP formulation of the BPP
def create_bin_packing_ilp(weights, bin_capacity):
    model = Model('BinPacking')
    num_items = len(weights)
    num_bins = num_items  # Upper bound on number of bins
    y = model.binary_var_list(num_bins, name="y")
    x = model.binary_var_matrix(num_items, num_bins, name="x")

    # Objective: Minimize number of bins used
    model.minimize(model.sum(y))

    # Constraints: Each item must be assigned to exactly one bin
    for i in range(num_items):
        model.add_constraint(model.sum(x[i, j] for j in range(num_bins)) == 1, f"item_{i}_assignment")

    # Constraints: Bin capacity must not be exceeded
    for j in range(num_bins):
        model.add_constraint(
            model.sum(weights[i] * x[i, j] for i in range(num_items)) <= bin_capacity * y[j],
            f"bin_{j}_capacity"
        )

    return model

# Create a function to transform the ILP model into a QUBO
def ilp_to_qubo(ilp_model):
    qp = from_docplex_mp(ilp_model)
    qp_qubo = QuadraticProgramToQubo().convert(qp)
    return qp_qubo

# Test functions with specific instances (size small, medium, and large)
def test_ilp_to_qubo(weights_list, bin_capacity):
    for weights in weights_list:
        print(f"\nProblem instance with weights: {weights}")
        ilp_model = create_bin_packing_ilp(weights, bin_capacity)
        qp_qubo = ilp_to_qubo(ilp_model)
        print("QUBO formulation:")
        print(qp_qubo.prettyprint())

In [5]:
# Test functions with specific instances
smallest_ilp_model = create_bin_packing_ilp(smallest_weights, bin_capacity)
small_ilp_model = create_bin_packing_ilp(small_weights, bin_capacity)
medium_ilp_model = create_bin_packing_ilp(medium_weights, bin_capacity)
large_ilp_model = create_bin_packing_ilp(large_weights, bin_capacity)

smallest_qp_qubo = ilp_to_qubo(smallest_ilp_model)
small_qp_qubo = ilp_to_qubo(small_ilp_model)
medium_qp_qubo = ilp_to_qubo(medium_ilp_model)
large_qp_qubo = ilp_to_qubo(large_ilp_model)

weights_list = [small_weights, medium_weights, large_weights]
test_ilp_to_qubo(weights_list, bin_capacity)



Problem instance with weights: [2, 3, 4]
QUBO formulation:
Problem name: BinPacking

Minimize
  4*bin_0_capacity@int_slack@0^2
  + 16*bin_0_capacity@int_slack@0*bin_0_capacity@int_slack@1
  + 16*bin_0_capacity@int_slack@0*bin_0_capacity@int_slack@2
  + 16*bin_0_capacity@int_slack@1^2
  + 32*bin_0_capacity@int_slack@1*bin_0_capacity@int_slack@2
  + 16*bin_0_capacity@int_slack@2^2 + 4*bin_1_capacity@int_slack@0^2
  + 16*bin_1_capacity@int_slack@0*bin_1_capacity@int_slack@1
  + 16*bin_1_capacity@int_slack@0*bin_1_capacity@int_slack@2
  + 16*bin_1_capacity@int_slack@1^2
  + 32*bin_1_capacity@int_slack@1*bin_1_capacity@int_slack@2
  + 16*bin_1_capacity@int_slack@2^2 + 4*bin_2_capacity@int_slack@0^2
  + 16*bin_2_capacity@int_slack@0*bin_2_capacity@int_slack@1
  + 16*bin_2_capacity@int_slack@0*bin_2_capacity@int_slack@2
  + 16*bin_2_capacity@int_slack@1^2
  + 32*bin_2_capacity@int_slack@1*bin_2_capacity@int_slack@2
  + 16*bin_2_capacity@int_slack@2^2 + 16*x_0_0*bin_0_capacity@int_slack@0
  +

## 2) **Create a Brute Force Solver for the QUBO Problem**
   - Solve the specific instances using a brute-force approach.

In [6]:
# Create a Brute Force solver for the QUBO problem and solve the specific instances.
def solve_qubo_brute_force(qubo, timeout=10):
    num_vars = qubo.get_num_binary_vars()
    best_energy = np.inf
    best_solution = None

    # Enumerate all possible combinations with progress bar
    total_combinations = 2 ** num_vars
    iterator = tqdm(itertools.product([0, 1], repeat=num_vars), total=total_combinations, desc="Brute Force Progress")

    start_time = time.time()
    check_time = start_time + timeout  # Time after which to check remaining time

    for binary_vector in iterator:
        binary_vector = np.array(binary_vector)  # Convert tuple to numpy array
        energy = qubo.objective.evaluate(binary_vector)
        if energy < best_energy:
            best_energy = energy
            best_solution = binary_vector

        current_time = time.time()
        estimated_remaining_time = 0
        if current_time >= check_time:
            # Check estimated remaining time
            elapsed_time = current_time - start_time
            iterations_completed = iterator.n
            if iterations_completed == 0:
                continue  # Avoid division by zero
            rate = iterations_completed / elapsed_time
            estimated_total_time = total_combinations / rate
            estimated_remaining_time = estimated_total_time - elapsed_time
            if estimated_remaining_time > 60:
                print("\nBrute Force computation estimated to take too long. Stopping early.")
                break  # Break the loop

    return best_solution, best_energy, estimated_remaining_time + current_time - start_time 

# Function to interpret the solution and extract item assignments to bins
def interpret_solution(solution, qubo):
    if solution is None:
        return None  # No solution found
    variables = qubo.variables
    x_vars = {}
    y_vars = {}
    for idx, var in enumerate(variables):
        var_name = var.name
        if var_name.startswith('x_'):
            x_vars[var_name] = solution[idx]
        elif var_name.startswith('y_'):
            y_vars[var_name] = solution[idx]

    num_bins = len(y_vars)
    items_in_bins = [[] for _ in range(num_bins)]
    for var_name, value in x_vars.items():
        if value == 1:
            _, i_str, j_str = var_name.split('_')
            i = int(i_str)
            j = int(j_str)
            items_in_bins[j].append(i)

    # Only include bins that are used
    bins_used = [j for j, y_val in enumerate(y_vars.values()) if y_val == 1]
    items_in_bins_used = [items_in_bins[j] for j in bins_used]

    return items_in_bins_used

def print_bins(items_in_bins, weights):
    if items_in_bins is None:
        print("No valid solution found.")
        return
    for bin_index, items in enumerate(items_in_bins):
        if items:
            bin_contents = [weights[item] for item in items]
            print(f"Bin {bin_index + 1}: Items {items}, Weights {bin_contents}, Total Weight: {sum(bin_contents)}")

In [7]:
# Solve smallest qubo using brute force
brute_solution, brute_energy, estimated_time = solve_qubo_brute_force(smallest_qp_qubo, timeout=20)
print("\nSmallest QUBO Brute Force Solution:")
if brute_solution is not None:
    items_in_bins = interpret_solution(brute_solution, smallest_qp_qubo)
    print_bins(items_in_bins, smallest_weights)
else:
    print("No valid solution found.")
print(f"Time taken: {estimated_time:.4f} seconds")

Brute Force Progress: 100%|██████████| 4096/4096 [00:01<00:00, 3495.30it/s]


Smallest QUBO Brute Force Solution:
Bin 1: Items [0, 1], Weights [2, 3], Total Weight: 5
Time taken: 1.1708 seconds


In [8]:
# Solve small qubo using brute force
brute_solution, brute_energy, estimated_time = solve_qubo_brute_force(small_qp_qubo, timeout=20)
bf_time = estimated_time
print("\nSmall QUBO Brute Force Solution:")
if brute_solution is not None:
    items_in_bins = interpret_solution(brute_solution, small_qp_qubo)
    print_bins(items_in_bins, smallest_weights)
else:
    print("No valid solution found.")
print(f"Time taken: {bf_time:.4f} seconds")

Brute Force Progress:   2%|▏         | 51868/2097152 [00:20<13:08, 2593.36it/s]


Brute Force computation estimated to take too long. Stopping early.

Small QUBO Brute Force Solution:
Time taken: 808.9271 seconds


## 3) **Solve the QUBO Using Quantum Annealing Simulators**
   - Use the [Dwave Ocean Framework](https://docs.ocean.dwavesys.com/en/stable/) to solve the QUBO. Refer to this [example](https://www.dwavesys.com/media/fmtj2fw3/20210920_ofbguide.pdf) for guidance.


In [9]:
from dimod import BinaryQuadraticModel, SimulatedAnnealingSampler

# Solve the QUBO using the simulated annealing solver
def solve_qubo_simulated_annealing(qubo, num_reads=100):
    # Convert QUBO to BinaryQuadraticModel (BQM)
    linear = qubo.objective.linear.to_dict()
    quadratic = qubo.objective.quadratic.to_dict()
    offset = qubo.objective.constant

    # Create a BinaryQuadraticModel (BQM)
    bqm = BinaryQuadraticModel(linear, quadratic, offset, vartype='BINARY')

    # Use the simulated annealing sampler from D-Wave's Neal package
    sampler = SimulatedAnnealingSampler()
    sampleset = sampler.sample(bqm, num_reads=num_reads)

    # Extract the best solution
    best_solution = sampleset.first.sample
    best_energy = sampleset.first.energy

    return best_solution, best_energy

qp_qubo = small_qp_qubo
st = time.time()
best_solution, best_energy = solve_qubo_simulated_annealing(qp_qubo)
qa_time = time.time() - st

items_in_bins = interpret_solution(list(best_solution.values()), qp_qubo)
print("\nQuantum Annealing Solution:")
print_bins(items_in_bins, small_weights)
print(f"Time taken: {qa_time:.4f} seconds")


Quantum Annealing Solution:
Bin 1: Items [0, 1], Weights [2, 3], Total Weight: 5
Bin 2: Items [2], Weights [4], Total Weight: 4
Time taken: 5.4291 seconds


## 4) **Use a Quantum Variational Approach to Solve the QUBO**
   - Create multiple [Ansatz](https://pennylane.ai/qml/glossary/circuit_ansatz/) for testing.
   - Build a function with the input being the QUBO and Ansatz. Use a hybrid approach to solve the QUBO.


In [10]:
# Use a Quantum Variational approach to solve the QUBO.
def solve_qubo_variational(qubo, ansatz):
    algorithm_globals.random_seed = 99
    maxiter = 100  # Set a maximum number of iterations

    num_qubits = qubo.get_num_binary_vars()

    # Convert QUBO to Ising Hamiltonian
    qubit_op, offset = qubo.to_ising()

    # Define the parameter vector for the ansatz
    num_parameters = ansatz.num_parameters
    parameters = ParameterVector('theta', num_parameters)

    # Assign parameters to the ansatz
    ansatz.assign_parameters(parameters, inplace=True)

    # Objective function to minimize
    def objective_function(param_values):
        param_dict = dict(zip(parameters, param_values))
        qc = ansatz.assign_parameters(param_dict)
        qc.measure_all()

        # Run the circuit
        sampler = Sampler()
        result = sampler.run([qc], shots=1024).result()
        counts = result.quasi_dists[0].binary_probabilities()

        # Compute expectation value
        energy = 0.0
        for bitstring, prob in counts.items():
            x = np.array([int(bit) for bit in bitstring[::-1]])
            value = qubo.objective.evaluate(x)
            energy += prob * value

        return energy

    # Initial parameters
    initial_params = np.random.rand(num_parameters)

    # Initialize progress bar
    progress_bar = tqdm(total=maxiter, desc="Variational Optimization Progress", leave=False)

    # Define callback function to update progress bar
    def callback(xk):
        progress_bar.update(1)

    # Set up the optimizer
    optimizer = COBYLA(maxiter=maxiter, callback=callback)

    # Optimize the parameters
    result = optimizer.minimize(fun=objective_function, x0=initial_params)
    progress_bar.close()

    # Get the optimal parameters
    optimal_params = result.x

    # Build the final circuit with optimal parameters
    param_dict = dict(zip(parameters, optimal_params))
    qc = ansatz.assign_parameters(param_dict)
    qc.measure_all()

    # Run the circuit and get the measurement results
    sampler = Sampler()
    result = sampler.run([qc], shots=1024).result()
    counts = result.quasi_dists[0].binary_probabilities()

    # Find the most probable bitstring
    max_bitstring = max(counts, key=counts.get)
    solution = np.array([int(bit) for bit in max_bitstring[::-1]])

    return solution

# Create multiple Ansatz for tests.
def create_ansatz_circuits(num_qubits):
    ansatz_list = []
    # Ansatz 1: Simple layer of Ry rotations
    ansatz1 = EfficientSU2(num_qubits, reps=2)
    ansatz_list.append(ansatz1)

    # Ansatz 2: TwoLocal circuit
    ansatz2 = TwoLocal(num_qubits, ['ry', 'rz'], 'cx', reps=2)
    ansatz_list.append(ansatz2)

    return ansatz_list

# Build a function with input being the QUBO and Ansatz. Using a hybrid approach solved the QUBO.
def hybrid_solver(qubo, ansatz_list, weights):
    times = []
    for idx, ansatz in enumerate(ansatz_list):
        print(f"\nUsing Ansatz {idx + 1}:")
        start_time = time.time()
        solution = solve_qubo_variational(qubo, ansatz)
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(solution)
        items_in_bins = interpret_solution(solution, qubo)
        print_bins(items_in_bins, weights)
        print(f"Time taken: {elapsed_time:.4f} seconds")
        times.append(elapsed_time)
    return times

In [11]:
# Use Quantum Variational approach with multiple Ansatz
print("\nQuantum Variational Approach Solutions:")
st = time.time()
ansatz_list = create_ansatz_circuits(qp_qubo.get_num_binary_vars())
var_time = hybrid_solver(small_qp_qubo, ansatz_list, small_weights)
for t in var_time:
  print(f"Total time taken: {t:.4f} seconds")


Quantum Variational Approach Solutions:

Using Ansatz 1:


C:\Users\natch\AppData\Local\Temp\ipykernel_36380\2267392761.py:93: DeprecationWarning: Sampler has been deprecated as of Aer 0.15, please use SamplerV2 instead.
  solution = solve_qubo_variational(qubo, ansatz)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Time taken: 42.5407 seconds

Using Ansatz 2:


C:\Users\natch\AppData\Local\Temp\ipykernel_36380\2267392761.py:93: DeprecationWarning: Sampler has been deprecated as of Aer 0.15, please use SamplerV2 instead.
  solution = solve_qubo_variational(qubo, ansatz)


[0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
Time taken: 68.6844 seconds
Total time taken: 42.5407 seconds
Total time taken: 68.6844 seconds



## 5) **Use [QAOA](https://arxiv.org/abs/1411.4028) to Solve the QUBO**
   - Create a QAOA function from scratch.

In [12]:
# Use QAOA to solve the QUBO.
def solve_qubo_qaoa(qubo):
    algorithm_globals.random_seed = 99
    maxiter = 100  # Set a maximum number of iterations

    # Custom QAOA implementation similar to Step 10
    num_qubits = qubo.get_num_binary_vars()
    p = 1  # Number of QAOA layers

    # Define parameters
    gamma = [Parameter(f'gamma_{i}') for i in range(p)]
    beta = [Parameter(f'beta_{i}') for i in range(p)]

    # Convert QUBO to Ising Hamiltonian
    qubit_op, offset = qubo.to_ising()

    # Create the parameterized QAOA circuit
    def create_qaoa_circuit(params):
        gamma_vals = params[:p]
        beta_vals = params[p:]

        qc = QuantumCircuit(num_qubits)

        # Initial state: apply Hadamard gates to all qubits
        qc.h(range(num_qubits))

        for layer in range(p):
            # Cost Hamiltonian evolution
            for idx, coeff in enumerate(qubit_op.coeffs):
                pauli_string = qubit_op.paulis[idx]
                label = pauli_string.to_label()
                z_indices = [i for i, pauli_char in enumerate(label) if pauli_char == 'Z']
                if len(z_indices) == 1:
                    # Single Z term
                    qc.rz(2 * coeff.real * gamma_vals[layer], z_indices[0])
                elif len(z_indices) == 2:
                    # ZZ interaction
                    qc.cx(z_indices[0], z_indices[1])
                    qc.rz(2 * coeff.real * gamma_vals[layer], z_indices[1])
                    qc.cx(z_indices[0], z_indices[1])

            # Mixer Hamiltonian evolution
            for q in range(num_qubits):
                qc.rx(2 * beta_vals[layer], q)

        return qc

    # Objective function to minimize
    def objective_function(params):
        qc = create_qaoa_circuit(params)
        qc.measure_all()

        # Run the circuit
        sampler = Sampler()
        result = sampler.run([qc], shots=1024).result()
        counts = result.quasi_dists[0].binary_probabilities()

        # Compute expectation value
        energy = 0.0
        for bitstring, prob in counts.items():
            x = np.array([int(bit) for bit in bitstring[::-1]])
            value = qubo.objective.evaluate(x)
            energy += prob * value

        return energy

    # Initial parameters
    initial_params = np.random.rand(2 * p)

    # Initialize progress bar
    progress_bar = tqdm(total=maxiter, desc="QAOA Optimization Progress", leave=False)

    # Define callback function to update progress bar
    def callback(xk):
        progress_bar.update(1)

    # Set up the optimizer
    optimizer = COBYLA(maxiter=maxiter, callback=callback)

    # Optimize the parameters
    result = optimizer.minimize(fun=objective_function, x0=initial_params)
    progress_bar.close()

    # Get the optimal parameters
    optimal_params = result.x

    # Build the final circuit with optimal parameters
    qc = create_qaoa_circuit(optimal_params)
    qc.measure_all()

    # Run the circuit and get the measurement results
    sampler = Sampler()
    result = sampler.run([qc], shots=1024).result()
    counts = result.quasi_dists[0].binary_probabilities()

    # Find the most probable bitstring
    max_bitstring = max(counts, key=counts.get)
    solution = np.array([int(bit) for bit in max_bitstring[::-1]])

    return solution

In [13]:
# Solve using QAOA
start_time = time.time()
solution = solve_qubo_qaoa(qp_qubo)
end_time = time.time()
qaoa_time = end_time - start_time
items_in_bins = interpret_solution(solution, small_qp_qubo)
print("\nQAOA Solution:")
print_bins(items_in_bins, small_weights)
print(f"Time taken: {qaoa_time:.4f} seconds")


QAOA Solution:
Bin 1: Items [0], Weights [2], Total Weight: 2
Time taken: 12.7199 seconds


C:\Users\natch\AppData\Local\Temp\ipykernel_36380\430475856.py:3: DeprecationWarning: Sampler has been deprecated as of Aer 0.15, please use SamplerV2 instead.
  solution = solve_qubo_qaoa(qp_qubo)
